In [ ]:
import h5py
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

# Load datasets
with h5py.File('train_catvnoncat.h5', 'r') as train_data:
    train_X = np.array(train_data['train_set_x'])
    train_Y = np.array(train_data['train_set_y'])
    
with h5py.File('test_catvnoncat.h5', 'r') as test_data:
    test_X = np.array(test_data['test_set_x'])
    test_Y = np.array(test_data['test_set_y'])

# Normalize the data
train_X = train_X / 255.0
test_X = test_X / 255.0

# Reshape for PyTorch compatibility
train_X = train_X.reshape(train_X.shape[0], -1)  # Flatten
test_X = test_X.reshape(test_X.shape[0], -1)
train_Y = train_Y.reshape(-1, 1)
test_Y = test_Y.reshape(-1, 1)

# Convert to PyTorch tensors
train_X = torch.tensor(train_X, dtype=torch.float32)
train_Y = torch.tensor(train_Y, dtype=torch.float32)
test_X = torch.tensor(test_X, dtype=torch.float32)
test_Y = torch.tensor(test_Y, dtype=torch.float32)

# Create PyTorch DataLoader
train_dataset = TensorDataset(train_X, train_Y)
test_dataset = TensorDataset(test_X, test_Y)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the model
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

model = LogisticRegressionModel(input_dim=train_X.shape[1])

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for X_batch, Y_batch in train_loader:
        predictions = model(X_batch)
        loss = criterion(predictions, Y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# Evaluate on test set
model.eval()
with torch.no_grad():
    predictions = (model(test_X) > 0.5).float()
    test_accuracy = (predictions == test_Y).float().mean().item()
    print(f"Test Accuracy: {test_accuracy:.4f}")

#indices of incorrect predictions
incorrect_indices = torch.where(predictions.flatten() != test_Y.flatten())[0].numpy()
print(f"Incorrect Predictions Indices: {incorrect_indices}")

# failed cases
plt.figure(figsize=(10, 10))
for i, idx in enumerate(incorrect_indices[:4]):
    plt.subplot(2, 2, i + 1)
    plt.imshow(test_X[idx].reshape(64, 64, 3).numpy())
    plt.title(f"Index: {idx}, Pred: {int(predictions[idx])}, True: {int(test_Y[idx])}")
    plt.axis('off')
plt.show()